In [ ]:
import pandas as pd


In [3]:
df_clincal_data = pd.read_csv("../../data/raw/brca_tcga_pub2015_clinical_data.tsv", sep='\t')
print(df_clincal_data.columns)

Index(['Study ID', 'Patient ID', 'Sample ID', 'Diagnosis Age',
       'American Joint Committee on Cancer Metastasis Stage Code',
       'Neoplasm Disease Lymph Node Stage American Joint Committee on Cancer Code',
       'Neoplasm Disease Stage American Joint Committee on Cancer Code',
       'American Joint Committee on Cancer Publication Version Type',
       'American Joint Committee on Cancer Tumor Stage Code',
       'Brachytherapy first reference point administered total dose',
       ...
       'Somatic Status', 'Staging System', 'Staging System.1',
       'Surgery for positive margins', 'Surgery for positive margins other',
       'Surgical procedure first', 'Tissue Source Site', 'TMB (nonsynonymous)',
       'Person Neoplasm Status', 'Tumor Disease Anatomic Site'],
      dtype='object', length=110)


In [4]:
df_clincal_data["Cancer Type"].unique()

array(['Invasive Breast Carcinoma'], dtype=object)

In [ ]:
df_clincal_data["ER Status By IHC"].head(20)

0     Positive
1     Positive
2     Positive
3     Positive
4     Negative
5     Positive
6     Positive
7     Negative
8     Positive
9     Positive
10    Negative
11    Positive
12    Positive
13    Negative
14    Negative
15    Positive
16    Negative
17    Negative
18    Positive
19    Negative
Name: ER Status By IHC, dtype: object

In [ ]:
df_clincal_data["PR status by ihc"].head(10)

0    Negative
1    Positive
2    Positive
3    Positive
4    Positive
5    Positive
6    Positive
7    Negative
8    Negative
9    Positive
Name: PR status by ihc, dtype: object

In [14]:
df_clincal_data["HER2 ihc score"].dropna().head(20)

2     1.0
4     2.0
6     2.0
7     0.0
8     3.0
10    0.0
14    2.0
15    2.0
16    2.0
17    1.0
18    2.0
20    3.0
21    1.0
23    1.0
24    0.0
27    2.0
29    2.0
30    2.0
31    2.0
32    1.0
Name: HER2 ihc score, dtype: float64

In [20]:
df_clincal_data["HER2 fish status"].head(20)

0          NaN
1          NaN
2     Negative
3          NaN
4     Negative
5          NaN
6     Negative
7          NaN
8     Positive
9          NaN
10         NaN
11         NaN
12    Negative
13    Negative
14    Negative
15    Negative
16    Negative
17    Positive
18    Negative
19    Positive
Name: HER2 fish status, dtype: object

In [19]:
df_clincal_data["HER2 ihc score"].head(20)

0     NaN
1     NaN
2     1.0
3     NaN
4     2.0
5     NaN
6     2.0
7     0.0
8     3.0
9     NaN
10    0.0
11    NaN
12    NaN
13    NaN
14    2.0
15    2.0
16    2.0
17    1.0
18    2.0
19    NaN
Name: HER2 ihc score, dtype: float64

In [ ]:
def classify_cancer_type(df_clinical_data : pd.DataFrame) -> str:
    ER_molecule = df_clincal_data["ER Status By IHC"]
    PR_molecule = df_clincal_data["PR status by ihc"]
    HER2_molecule = df_clincal_data["HER2 fish status"]
    HER2_molecule_score = df_clincal_data["HER2 ihc score"]
    
    
    
    return "hello world"